In [125]:
from utils import *
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [107]:
X,y = load_rec_talk_mix()

In [108]:
np.unique(y)

array([-1.,  1.])

In [109]:
kfold = KFold(n_splits = 10, shuffle = True,random_state = 41)

In [110]:
vectorizer = TfidfVectorizer()

In [111]:
X= vectorizer.fit_transform(X)

In [11]:
X.shape

(7232, 67710)

In [7]:
scores = []
for train_index,test_index in kfold.split(X):
#     print((train_index).shape,(test_index).shape)
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    clf = SVC(kernel = 'linear')
    clf.fit(X_train,y_train)
    scores.append(score_acc(clf,X_test,y_test))

In [10]:
# scores # 这结果要比论文中的好多了吧，识别率100%

## 线性支持向量机最优化问题
$$min_{w,b} \frac{1}{2} ||w||^{2}\\ s.t. y_{i}(w*x_{i}+b)-1>=0,i=1,2,3,N$$

## 凸优化问题
$$min_{w} f(w) \\ s.t. g_{i}(x)<=0 \\ h_{i}(w) = 0$$
- 其中，目标函数$f(w)$和约束函数$g_{i}(w)$都是$R_{n}$上的连续可微的凸函数，约束函数$h_{i}(w)$是$R^{n}$上的仿射函数
- 仿射函数 $f(x)$为仿射函数,如果他满足$f(x)=ax+b,a\in R^{n},b\in R,x \in R^{n}$

## 学习的对偶算法
- 应用拉格朗日对偶性，通过求解对偶问题得到原始问题的解
- 对偶问题往往更加容易求解
- 自然引入核函数，进而推广到非线性分类问题


## 构建拉格朗日函数
$$L(w,b,a)=\frac{1}{2}||w||^{2}-\sum_{i=1}^N{\alpha_{i}y_{i}(wx_{i}+b)}+\sum_{i=1}^{N}{\alpha_{i}}\tag{1.1}$$
- 是根据凸优化问题构建的
- 原始问题求解的是$$min_{\alpha}max_{w,b}L(w,b,\alpha)$$ 只有在满足<b>约束条件</b>求最大值的时候才是原来的方程
- 对偶问题$$max_{\alpha}min_{w,b}L(w,b,\alpha)$$

## 线性可分求解过程
- 将$\alpha$看做是一个固定值，分别对$w$和$b$求导得到
$$\begin{eqnarray*}w =\sum_{i=1}^{N}{\alpha_{i}y_{i}x_{i}}\tag{1.2}\\ 
\sum_{i=1}^{N}{\alpha_{i}y_{i}}=0\tag{1.3} \end{eqnarray*}$$
- 将式1-2代入到1-1中得到
$$min_{w,b}L(w,b,a)=-\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}{\alpha_{i}\alpha_{j}y_{i}y_{j}(x_{i}*x_{j})}+\sum_{i=1}^{N}{\alpha_{i}}$$
- 对$min_{w,b}L(w,b,\alpha)$对$\alpha$的极大。即是
$$max_{\alpha}-\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}{\alpha_{i}\alpha_{j}y_{i}y_{j}(x_{i}*x_{j})}+\sum_{i=1}^{N}{\alpha_{i}} \tag{1.4}\\ 
s.t. \sum_{i=1}^{N}{\alpha_{i}y_{i}}=0 \\
\alpha_{i}>=0
$$
- 式1.4的对偶问题为求最小值，然后使用SMO算法就可以求解$\alpha$
- 求出最优化的$\alpha^{*}$之后，代入1.3求出$w$，找到一个$\alpha_{j}>0$, 
$b^{*}=y_{j}-\sum_{i=1}^{N}{\alpha_{i}^{*}y_{i}(x_{i}x_{j})}$
- 最后的求解函数为
$$f(x)=sign(\sum_{i=1}^{N}\alpha_{i}y_{i}K(x,x_{i})+b^{*})$$

## 软间隔最大化
- 线性不可分意味着某些样本点$(x_{i},y_{i})$不能满足函数间隔大于等于1的约束条件。
- 其原始问题为：
$$min_{w,b,\xi}\frac{1}{2}||w||^{2}+C\sum_{i=1}^{N}\xi_{i}\\
s.t. y_{i}(wx_{i}+b)>=1-\xi_{i}, i =1,2,...,N \\
\xi_{i}>=0, i=1,2,...,N
$$
- 软间隔的支持向量$x_{i}$或者在间隔边界上，或者在间隔边界与分离超平面之间，或者在分离超平面误分一侧。若$\alpha_{i}<C,则\xi_{i}=0$,支持向量$x_{i}$恰好落在间隔边界上；若$\alpha_{i}=C,0<\xi_{i}<1$,则分类正确，$x_{i}$在间隔边界与分离超平面之间；若$\alpha_{i}=C,\xi_{i}=1$,则$x_{i}$在分离超平面上；若$\alpha_{i}=C,\xi_{i}>1$,则$x_{i}$位于分离超平面误分类一侧。

## 核函数
- 设$\chi$是输入空间，又称$H$为特征空间，如果存在一个从$\chi$到$H$的映射
$F(x):\chi \to H$使得对所有$x,z\in \chi$，函数$K(x,z)$满足条件$K(x,z)=F(x)F(z)$,则称$K(x,z)$为核函数，$F(x)$为映射函数


## 正定核函数的判断？

## 序列最小最优化算法SMO算法
- 是用来求解1.4中的算法
- 其是一种启发式算法，其基本思路是:如果两个变量的解都满足此优化问题的KTT条件，那么这个最优化问题的解就得到了。
- 求解目标
$$min_{\alpha} \frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}{\alpha_{i}\alpha_{j}y_{i}y_{j}(x_{i}*x_{j})}-\sum_{i=1}^{N}{\alpha_{i}} \tag{1.5}\\ 
s.t. \sum_{i=1}^{N}{\alpha_{i}y_{i}}=0 \\
0=<\alpha_{i}<=C
$$
- 当训练样本全部满足KKT条件，训练结束

In [120]:
## ICDI SVM
class SVM(object):
    def __init__(self,C = 1.0, eps = 1e-3, num_iterator = 1000):
        self.C = C # 惩罚系数
        self.eps = eps # 精度,用于判断准确度
        self.b = 0
        self.num_iterator = num_iterator
    def kernel(self, x_i, x_j): # 默认使用的是线性核函数，即 F(x,y) = f(x)*f(y)
        return np.dot(self.train[x_i],self.train[x_j].transpose())
    
    # 求得的是预测的值
    def g(self,x_j):
        sum = 0
        for i,x_i in enumerate(self.train):
            if self.cmp(self.alpha[i]) == 0: # 为零
                continue
            sum += self.alpha[i] * self.target[i] * self.kernel(i,x_j);
        return sum + self.b
    
    # 求得是预测值和真实值之间的差值
    def E_fun(self, x_i):
        return self.g(x_i) - self.target[x_i]
    
    def cmp(self, x):
        if abs(x) < self.eps:
            return 0
        if x > 0:
            return 1
        return -1;
    
    # 判断是否满足KTT 条件 ，这个地方写的很好
    def _satisfy_KKT(self, i):
        ygx = self.target[i] * self.g(i)
        if abs(self.alpha[i])<self.eps:
            return ygx > 1 or ygx == 1
        elif abs(self.alpha[i]-self.C)<self.eps:
            return ygx < 1 or ygx == 1
        else:
            return abs(ygx-1) < self.eps
    
    # 更新alpha值
    def update_alpha(self,x_i, x_j): # i为2，j 为1
        if self.target[x_i] != self.target[x_j]:
            L = max(0,self.alpha[x_i] - self.alpha[x_j])
            H = min(self.C,self.C + self.alpha[x_i] - self.alpha[x_j])
        if self.target[x_i] == self.target[x_j]:
            L = max(0,self.alpha[x_i] + self.alpha[x_j] - self.C)
            H = min(self.C, self.alpha[x_i] + self.alpha[x_j])
        alpha_x_i_old = self.alpha[x_i]
        alpha_x_j_old = self.alpha[x_j]
        
        eta = self.kernel(x_i,x_i) + self.kernel(x_j,x_j) - 2 * self.kernel(x_i,x_j)
       
        self.alpha[x_i] = self.alpha[x_i] + self.target[x_i] * (self.E[x_j] - self.E[x_i]) / eta
        if self.alpha[x_i] > H:
            self.alpha[x_i] = H
        elif self.alpha[x_i] < L:
            self.alpha[x_i] = L
        
        self.alpha[x_j] = self.alpha[x_j] + self.target[x_i] * self.target[x_j] * (alpha_x_i_old - self.alpha[x_i]) 
        
        
        b1_new = - self.E[x_j] - self.target[x_j] * self.kernel(x_j,x_j) * (self.alpha[x_j] - alpha_x_j_old) - self.target[x_i] * self.kernel(x_i,x_j) * (self.alpha[x_i] - alpha_x_i_old) + self.b
        b2_new = - self.E[x_i] - self.target[x_j] * self.kernel(x_i,x_j) * (self.alpha[x_j] - alpha_x_j_old) - self.target[x_i] * self.kernel(x_i,x_i) * (self.alpha[x_i] - alpha_x_i_old) + self.b
       
        if 0 < self.alpha[x_j] and self.alpha[x_j] < self.C:
            self.b = b1_new
        elif 0 < self.alpha[x_i] and self.alpha[x_i] < self.C:
            self.b = b2_new
        else:
            self.b = (b1_new + b2_new ) / 2
        self.E[x_i] = self.E_fun(x_i)
        self.E[x_j] = self.E_fun(x_j)
    
    def choice_i_j(self):
        # 直接找违反KTT 条件的值
        index_list = [i for i in range(self.N)]

        j1_list_1 = list(filter(lambda i: self.alpha[i] > 0 and self.alpha[i] < self.C, index_list))
        j1_list_2 = list(set(index_list) - set(j1_list_1))

        j1_list = j1_list_1
        j1_list.extend(j1_list_2)

        for j in j1_list:
            if self._satisfy_KKT(j):
                continue

            E1 = self.E[j]
            max_ = (0, 0)

            for i in index_list:
                if i == j:
                    continue

                E2 = self.E[i]
                if abs(E1 - E2) > max_[0]:
                    max_ = (abs(E1 - E2), i)

            return j, max_[1]
        return -1,-1
    
    def fit(self,X,Y):
        self.alpha = np.zeros(X.shape[0]) # 初始化每一个参数
        self.train = X
        self.target = Y
        self.N = X.shape[0]
        self.E = np.array([self.E_fun(i) for i in range(self.N)])
#         while(True):
        for i in range(self.num_iterator):
            print('iterator: ', i)
            j,i = self.choice_i_j()
            if i == -1 and j == -1:
                break
            self.update_alpha(i,j)
    def predict(self,X):
        result = []
        for x in X:
            sum = 0
            for i,x_i in enumerate(self.train):
                if self.cmp(self.alpha[i]) == 0: # 为零
                    continue
                sum += self.alpha[i] * self.target[i] * np.dot(x.transpose(),x_i);
            result.append(self.cmp(sum + self.b))

                

In [124]:
scores_my = []
for train_index,test_index in kfold.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    clf = SVM()
    clf.fit(X_train.toarray(),y_train)
    score = score_acc(clf,X_test.toarray(),y_test)
    scores.append(score)

In [122]:
np.min(score)

1.0